# Artist Similarity with Graph Neural Network 1st Notebook

In this notebook are implemented the experiments that were previously conducted in the paper  ['Artist Similarity with Graph Neural Network'](https://archives.ismir.net/ismir2021/paper/000043.pdf).  
In addition to the architectures described in the research there are also experiments with the famous Graph Attention layer, in order to show how this approach can sharply outperform the GraphSAGE configuration.  
In this notebook are shown the performances from the different embeddings of artists.

In [1]:
# !pip install torchmetrics
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch-geometric
# !pip install pytorch_metric_learning

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import pairwise_euclidean_distance
from torch_geometric.nn import GATConv, SAGEConv
from torch.optim import lr_scheduler
import random
from random import choice,randrange
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import math
import time
from torch_geometric import seed_everything
from sklearn.manifold import TSNE
import pandas as pd
from pytorch_metric_learning import losses
from utils import *
from sklearn.metrics import f1_score


random_seed=280085

seed_everything(random_seed)

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.0


In [3]:
import torch_geometric
print(torch_geometric.__version__)

2.3.0


In [4]:
from utils import *  #In this files are reported the most useful functions
from architectures import *

1.13.0


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


With the help of the [Torch geometric framework](https://pytorch-geometric.readthedocs.io/en/latest/) was really easy to handle the graph attributes and nodes and then the training of the GNNs.

In [6]:
random = False
if random == True:
  A1 = torch.load('adjacency').to(device)      # Normal adjacency matrix format is obtained with torch.load('adjacency')
  X = torch.load('random_instance').to(device)
else:
  X = torch.load('instance').T.to(device)      # Instance matrix
  A1 = torch.load('adjacency').to(device) 

A = torch.load('adjacencyCOO').to(device)    # Adjacency matrix in the COO format, that is that supported by torch geometric


label_diz = load_data('data/encode_labels.json') # Genre to idx
label_diz2 = {label_diz[key] : key for key in label_diz} # Idx to genre
artists = load_data('data/artist_genres.json') # Artist names

labels = torch.load('labels.pt') 
num_classes = torch.unique(labels).shape[0]

num_samples = X.shape[0]
print('The number of samples and classes is {} and {} respectively'.format(num_samples, num_classes))

The number of samples and classes is 11261 and 25 respectively


In [7]:
''' These variables contain the information about the artists' names, and their position in the dataset, this makes easy to look for their name and to better draw conclusions at inference time '''
num2artist = load_data('data/dizofartist.json')
artist2num = {num2artist[key]:key for key in num2artist}

# print(num2artist)
# print(artist2num)

## Import the data with Torch geometric:


In [8]:
''' In order to conduct the experiments was fundamental to split the dataset, either the nodes and also the edges.
    The splitting was performed according to the information in the paper, and considering the fact that a lot of date were lost in the preprocessing part.'''

from torch_geometric.data import Data
from torch_geometric.utils import structured_negative_sampling


''' This variable contains the indices for the splitting, that are necessary to compute the masks, according to the torch geometric pipeline '''
data_summary = {'train_with_val' : {'low' : 0, 'high': 9022},
                'train' : {'low' : 0, 'high' : 10190},
                'val' : {'low' : 9022, 'high' : 10190},
                'test' : {'low' : 10190, 'high' : 11261}}

total_mask = torch.zeros(X.shape[0], dtype = torch.bool)

vtrain_mask = total_mask.clone()
train_mask = total_mask.clone()
val_mask = total_mask.clone()
test_mask = total_mask.clone()
eval_val = total_mask.clone()


vtrain_mask[data_summary['train_with_val']['low']:data_summary['train_with_val']['high']] = True
val_mask[data_summary['val']['low']:data_summary['val']['high']] = 1
train_mask[data_summary['train']['low']:data_summary['train']['high']] = 1
test_mask[data_summary['test']['low']:data_summary['test']['high']] = 1

eval_val[data_summary['train_with_val']['low']:data_summary['val']['high']] = 1

kwargs = {'vtrain_mask':vtrain_mask, 'train_mask':train_mask, 'val_mask':val_mask, 'test_mask':test_mask}

data = Data(x=X, edge_index = A, y = labels, **kwargs)


class data_split:
  ''' This class shows an alternative to the torch geometric masks procedure, it was necessary at inference time, where was needed the whole graph for the embedding compuutation '''
  def __init__(self, data, low, high):
    ''' Starting from the index 0 to 11260, we choose the interval of intersting samples
        self.data: contains the whole dataset (nodes and edges)
        self.rang: define the boundaries
        self.get_split_from_bounds perform the splitting, returning a x and edge_index attribute resembling the torch geometric Data objects.'''
    self.data = data
    self.rang = torch.arange(low, high + 1,1, device = 'cuda' if torch.cuda.is_available() else 'cpu')
    self.get_split_from_bounds(low, high)

  def get_split_from_bounds(self, low, high):
    self.x= self.data.x[low:high]
    v1_0 = self.data.edge_index[0]
    v2_0 = self.data.edge_index[1]
    v1_1 = v1_0[v1_0 < high]
    v1_2 = v1_1[v1_1 >= low]

    v2_1 = v2_0[v1_0 < high]
    v2_2 = v2_1[v1_1 >= low]
    v2_3 = v2_2[v2_2 < high]
    v2_4 = v2_3[v2_3 >= low]
    v1_3 = v1_2[v2_2 < high]
    v1_4 = v1_3[v2_3 >= low]

    self.edge_index = torch.cat((v1_4.unsqueeze(0), v2_4.unsqueeze(0)), dim = 0)
    
    return self.x, self.edge_index

  def split_for_inference(self, low_train, low_test, high_train, high_test):
    ''' At inference time we need to compute the embedding through the train and test artists, but we don want to consider the linkings between the test artist, those must be predicted.
        This function takes as input the boundaries of the train, and test set, computes the edge indices by removing the undesired connection.
        This method will be used to compute the accuracy. '''
    
    final_edge_indices = torch.tensor([[],[]], device = 'cuda' if torch.cuda.is_available() else 'cpu')
    for edge in range(self.edge_index.shape[1]):
      up = self.edge_index[0][edge].item()
      down = self.edge_index[1][edge].item()

      if up in range(low_test,high_test) and down in range(low_test, high_test): # If the connection is between test artist we remove it from the edge indices.
        continue

      else:
        final_edge_indices = torch.cat((final_edge_indices, self.edge_index[:,edge].reshape((2,1))), dim = 1)

    if device.type == 'cuda':
      return self.x, final_edge_indices.type(torch.cuda.LongTensor)
    else:
      return self.x, final_edge_indices.type(torch.LongTensor)


  
# visualize(torch.randn((11261, 2)), labels)


In [9]:
def visualize(h, color, dim = 2):

    z = TSNE(n_components = dim).fit_transform(h.detach().cpu().numpy())
    # z = h
    if dim == 2:
        p = pd.DataFrame(columns = ['x1', 'x2', 'y'])
        p['x1'] = z[:, 0]
        p['x2'] = z[:, 1]
    elif dim == 3:
        p = pd.DataFrame(columns = ['x1', 'x2', 'x3', 'y'])
        p['x1'] = z[:, 0]
        p['x2'] = z[:, 1]
        p['x3'] = z[:, 2]


    p['y'] = [label_diz[str(int(num.item()))] for num in color]
    p['name'] = [artists[str(int(num))][1] for num in range(color.shape[0])]
    p['old genre'] = [[list(artists[str(int(num))][0][genre].values())[0] for genre in range(len(artists[str(int(num))][0]))] for num in range(color.shape[0])]


    if dim == 2:
        fig = px.scatter(p, x="x1", y="x2", color="y", hover_data = ['name', 'old genre'])
    elif dim == 3:
        fig = px.scatter_3d(p, x="x1", y="x2", z = "x3", color="y", hover_data = ['name', 'old genre'])
        fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

    fig.show()
    return z, p


## Define the architectures

* GraphSage-based architecture is the one defined in the original paper [Artist similarity with Graph Neural Networks](https://arxiv.org/pdf/2107.14541.pdf).  
  1. SAGEConv(2613,256)
  2. SAGEConv(256,256) *
  3. SAGEConv(256,256) *
  4. Linear(256,100)  
  5. TripletLoss(a,p,n)
  
The second and third layer are optionals based on the desired configuration.

* The GAT-based architectures are the one used for our own experiments, they are defined as follows:  

GAT1: 
  1. GATConv(2613, 256)    *Multi-head attention mechanism
  2. GATConv(256 * n_heads, 256) *Multi-head attention mechanism
  3. Linear(256 * n_heads, 256)  
  4. Linear(256, 256)
  5. TripletLoss(a,p,n)

GAT2:  
  1. Linear(2613, 256)
  2. Linear(256,256)
  3. Linear(256, 256) 
  4. GATConv(256 , 256) *Multi-head attention mechanism
  5. GATConv(256* n_heads, 256)    *Multi-head attention mechanism
  6. TripletLoss(a,p,n)

According to the paper's authors the training was done with the aid of mini-batches of 512 size, in order to do so the torch-geometric framework offer the NeighborLoader method, that shuffles the data and look for the neighbors of each batch in a few seconds.

In [10]:
from torch_geometric.loader import NeighborLoader

In [21]:
lamb = 0.8 # This is the value used for the distance weighted sampling

def get_triplets(embedding, edges, batch_size, get_vecs = False):
  ''' The loss function to minimize is  a triplet loss function, thus we need to look for positives and negatives for each sample in the batches.
      This function takes as input:
      * embedding:  the output of the GCN.
      * edges:      the edges for the mini-batch.
      * batch_size: the size for the batch.
      * get_vecs:   It is a boolean, if True the function returns the tensor of positives and negatives, otherwise only the indices are returned.  '''

  n_of_neigh = 10 # How many samples to consider for the sampling
  edges_n = edges.clone()
  edges[0] = edges_n[1]
  edges[1] = edges_n[0]
  
  total_triplets = structured_negative_sampling(edges)
  total_ancor = total_triplets[0]
  total_pos = total_triplets[1]
  total_negs = total_triplets[2]

 
  shape = batch_size
  positives = torch.zeros(shape)
  negatives = torch.zeros(shape)
  
  for ancor in range(shape):
    pos = total_pos[total_ancor == ancor] # We get the positives that are also neighbors for the anchor
    #pos = pos_n[pos_n < 512]  
    if pos.shape[0] > n_of_neigh:
      pos = pos[:n_of_neigh]
    neg = total_negs[total_ancor == ancor] # We get the negatives that are also neighbors for the anchor
    #neg = neg_n[neg_n < 512]  

    if neg.shape[0] > n_of_neigh:
      neg = neg[:n_of_neigh]
    p = 1
    n = 1

    if pos.shape[0] == 0:
      positives[ancor] == ancor
      p = 0
    
    if neg.shape[0] == 0:
      negatives[ancor] == ancor
      n = 0
    
    if p:
      pos_index = compute_idx_p(embedding, pos, ancor)
      positives[ancor] = pos_index
    if n:
      neg_index = compute_idx_n(embedding, neg, ancor)
      negatives[ancor] = neg_index
  
  
  if device.type == 'cuda':
    positives = positives.type(torch.cuda.LongTensor) 
    negatives = negatives.type(torch.cuda.LongTensor)
  else:
    positives = positives.type(torch.LongTensor)
    negatives = negatives.type(torch.LongTensor)

  
  if get_vecs:
    return embedding[positives], embedding[negatives] #Return the embedding vectors

  else:
    return positives, negatives                       # Or return the list of indices


def compute_idx_p(embedding, pos, ancor):
# This function performs the distance weighted sampling.
# We look for all positives for the anchor sample and we weight their distance from it. We choose then the 'hard positive' namely one of the furthest positives for the anchor
  diz_pos_ = {}
  #print(pos)
  for idx in pos:
    diz_pos_[idx.item()] = pairwise_euclidean_distance(embedding[ancor].unsqueeze(0), embedding[idx.item()].unsqueeze(0))[0][0].item()
  max_dist = max(list(diz_pos_.values())) if max(list(diz_pos_.values()))!=0 else 1e-5
  keys = list(diz_pos_.keys())

  for key in keys:
    if diz_pos_[key]/max_dist > lamb and len(diz_pos_) != 1:
      diz_pos_.pop(key)


  return max(diz_pos_,key=diz_pos_.get)

def compute_idx_n(embedding, neg, ancor):
# This function performs the distance weighted sampling.
# We look for all negatives for the anchor sample and we weight their distance from it. We choose then the 'hard negative' namely one of the closest negative for the anchor 
    diz_neg_ = {}
    #print(neg)
    for idx in neg:
      diz_neg_[idx.item()] = pairwise_euclidean_distance(embedding[ancor].unsqueeze(0), embedding[idx.item()].unsqueeze(0))[0][0].item()

  
    max_dist = max(list(diz_neg_.values())) if max(list(diz_neg_.values()))!=0 else 1e-5
    keys = list(diz_neg_.keys())
    for key in keys:
      if diz_neg_[key]/max_dist < 1 - lamb and len(diz_neg_) != 1:
        diz_neg_.pop(key)


    return min(diz_neg_,key=diz_neg_.get)


class Trainer:
  ''' This class contains all the method needed to train, test, evaluate, save and load the model.''' 
  def __init__(self, model, optimizer, scheduler, loss, num_epochs, mode, path, first, loss_mode = 'triplet'):

    self.model = model  # model class
    self.optimizer = optimizer # Adam optimizer
    self.scheduler = scheduler # Cosine learning rate scheduler
    self.loss = loss           # Triplet Loss function
    self.num_epochs = num_epochs # Num of epochs for the training
    self.mode = mode             # This must be specified to know what splitting to use in the model evaluation
    self.path = path             # This is the path where to save the model, if None the model won't be saved, otherwise it will be at the end of each epoch.
    self.first = first
    self.loss_mode = loss_mode

    if self.loss_mode == 'sphere':
      self.loss_optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = weight_decay)

    if self.loss_mode == 'triplet+':
      self.loss2 = nn.CrossEntropyLoss()
      self.predictor = Predictor().to(device)
      self.pred_opt = torch.optim.Adam(self.predictor.parameters(), lr = 0.001, weight_decay = weight_decay)

    if self.first != True:
      self.checkpoint = load_model(self.path, self.model, device) # If exists already a checkpoint for the model we load it.
    else:
      self.checkpoint = {}



  def train(self):
    self.model.train()

    for epoch in range(self.num_epochs):
      print("Processing {}-th epoch".format(epoch+1))
      print("Training step....")
      loss_train_list = []
      for batch in train_loader:
        self.optimizer.zero_grad()

        out = self.model(batch.x, batch.edge_index.to(device))
        out_l = out[:batch.batch_size]

        if self.loss_mode.startswith('triplet'):
          positives, negatives = get_triplets(out.clone(), batch.edge_index.clone(), batch.batch_size, get_vecs = True)
          loss_train = self.loss(out_l, positives, negatives)
          if self.loss_mode == 'triplet+':
            out_n = self.predictor(out_l)
            
            loss2 = self.loss2(out_n, batch.y[:batch.batch_size].to(device)) 

            loss_train +=  1 * loss2
          
        elif self.loss_mode == 'sphere':
          loss_train = self.loss(out_l, batch.y[:batch.batch_size].to(device))
          
        loss_train.backward()
        self.optimizer.step()
        if self.loss_mode == 'triplet+':
          self.pred_opt.step()
        if self.loss_mode == 'sphere':
          self.loss_optimizer.step()
        loss_train_list.append(loss_train.item())

      if self.loss_mode != 'triplet+':
        if self.loss_mode == 'sphere':
          self.loss_test_f, f1_s = self.test()
        else:
          self.loss_test_f = self.test()

      else:
        self.loss_test_f, self.loss_pred, f1_s = self.test()

      self.loss_train_f = sum(loss_train_list)/len(loss_train_list)
      self.accuracy_on_test = self.eval_accuracy(self.mode)

      self.end_epoch()
      self.scheduler.step()

      if self.loss_mode == 'triplet':
        print("At the {}-th epoch we have obtained: train_loss {:.6f} \t test_loss {:.6f} \t test_accuracy {:.6f}".format(epoch+1,self.loss_train_f, self.loss_test_f, self.accuracy_on_test))
        
      elif self.loss_mode == 'sphere':
        print("At the {}-th epoch we have obtained: train_loss {:.6f} \t test_loss {:.6f} \t test_accuracy {:.6f} \t f1_score {:.6f}".format(epoch+1,self.loss_train_f, self.loss_test_f, self.accuracy_on_test, f1_s))
      else:
        print("At the {}-th epoch we have obtained: train_loss {:.6f} \t test_loss {:.6f} \t test_loss_pred {:.6f} \t test_accuracy {:.6f} \t f1_score {:.6f}".format(epoch+1,self.loss_train_f, self.loss_test_f, self.loss_pred, self.accuracy_on_test, f1_s))

    
    return self.accuracy_on_test
    

  def test(self):
    self.model.eval()
    with torch.no_grad():
      print("Testing step....")
      loss_test_list = []
      loss_pred = []
      f1_list_pred = torch.tensor([])
      f1_list_true = torch.tensor([])

      for batch in test_loader:

        out = self.model(batch.x, batch.edge_index.to(device))
        out_l = out[:batch.batch_size]

        if self.loss_mode.startswith('triplet'):
            positives, negatives = get_triplets(out.clone(), batch.edge_index.clone(), batch.batch_size, get_vecs = True)
            loss_test = self.loss(out_l, positives, negatives)
            if self.loss_mode == 'triplet+':
              out_n = self.predictor(out_l)
              loss2 = self.loss2(out_n, batch.y[:batch.batch_size].to(device))
              loss_pred.append(loss2)
              # f1_s = self.compute_f1_score(batch.y[:batch.batch_size], out_n)
              f1_list_pred = torch.cat([f1_list_pred, torch.argmax(F.softmax(out_n, dim = -1), dim = -1)])
              f1_list_true = torch.cat([f1_list_true, batch.y[:batch.batch_size]])



        elif self.loss_mode == 'sphere':
          loss_test = self.loss(out_l, batch.y[:batch.batch_size].to(device))
          f1_list_pred = torch.cat([f1_list_pred, torch.argmax(F.softmax(out_l, dim = -1), dim = -1)])
          f1_list_true = torch.cat([f1_list_true, batch.y[:batch.batch_size]])
        
        loss_test_list.append(loss_test.item())

    if self.loss_mode != 'triplet+':

      if self.loss_mode == 'sphere':
        f1_s = self.compute_f1_score(f1_list_true, f1_list_pred)
        return sum(loss_test_list)/len(loss_test_list), f1_s
      
      return sum(loss_test_list)/len(loss_test_list) 
    else:
      f1_s = self.compute_f1_score(f1_list_true, f1_list_pred)
      return sum(loss_test_list)/len(loss_test_list), sum(loss_pred)/len(loss_pred), f1_s

  def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
    if ID>200:       # as described in the paper.
        ID=200
    c=1
    somm=0
    while c<=ID:
        somm+=1/(math.log2(1+c))
        c+=1
    return somm
  
  def compute_f1_score(self, true, pred, avg = 'macro'):

    pred = pred.detach().cpu().numpy()
    true = true.cpu().numpy()

    f1_s = f1_score(true, pred, labels = np.arange(num_classes), average = avg)

    return f1_s

  def eval_accuracy(self, mode = 1): 
    ''' This function computes the Normalized Discounted Cumulative Gain, that is the metric adopted in the research. '''
    if mode == 1:
      low_train = data_summary['train_with_val']['low']
      high_train = data_summary['train_with_val']['high']
      low_test = data_summary['val']['low']
      high_test = data_summary['val']['high']

    else:
      low_train = data_summary['train']['low']
      high_train = data_summary['train']['high']
      low_test = data_summary['test']['low']
      high_test = data_summary['test']['high']      


    self.model.eval()
    with torch.no_grad():
      ''' It is necessary to compute the embedding by condisering the edges between train and test data, but without considering the linkings between test samples, because they ,must be predicted in the evaluation. '''
      inference_data = data_split(data, low = low_train, high = high_test).split_for_inference(low_train, low_test, high_train, high_test) # This function takes care of the link remotion.
      out = self.model(inference_data[0], inference_data[1].to(device))[torch.arange(low_test,high_test)]

      A_acc = A1[low_test:high_test, low_test:high_test]

      print("Evaluation step....")
      test_embs = out.to(torch.device('cpu')).numpy()
      
      neigh=NearestNeighbors(n_neighbors=(200+1),algorithm='ball_tree').fit(test_embs) #With the K-NN we get the nearest 
      
      dist,ind=neigh.kneighbors(test_embs) 

      acc=[]

      c=0
      for test_artist in range(high_test-low_test):
          summ=0
          # ideal=len([i for i in range(self.test[0],self.test[0]+A_acc[k,:].shape[0]) if A_acc[k,i]!=0]) 
          
          ideal = A_acc[ind[c][0]].sum().item()
          
          
          
          den = self.calcG(ideal)
          if den==0:
              c+=1
              continue  
          for j in range(len(ind[c][1:])):
              if A_acc[ind[c][0]][ind[c][1:][j]].item()!=0:
                  summ+= 1/(math.log2(1+(j+1)))
                  
              else:
                  continue
          c+=1    
          summ/=den
          acc.append(summ)
      return sum(acc)/len(acc)

  def end_epoch(self):

    ''' This method is called at the end of each epoch and its purpose is to save the model state, and metrics, in order to be loaded again when needed.'''
    if self.path != None:
      if self.first:
        self.checkpoint['loss_train'] = [self.loss_train_f]
        self.checkpoint['loss_test'] = [self.loss_test_f]
        self.checkpoint['accuracy'] = [self.accuracy_on_test]
        self.checkpoint['modelState'] = self.model.state_dict()
        save_model(self.checkpoint, self.path)
        self.first = False
      else:
        self.checkpoint['loss_train'] += [self.loss_train_f]
        self.checkpoint['loss_test'] += [self.loss_test_f]
        self.checkpoint['accuracy'] += [self.accuracy_on_test]
        self.checkpoint['modelState'] = self.model.state_dict()
        save_model(self.checkpoint, self.path)

      load_model(self.path, self.model, device)



    

In [22]:
set_mode = 'test'  # 'val' or 'test', in order to set up the specific training procedure

data = data

if set_mode == 'val':
  trainmask = data.vtrain_mask
  testmask = data.val_mask
  mode = 1
elif set_mode == 'test':
  trainmask = data.train_mask
  testmask = data.test_mask
  mode = 2


#Gat2Model = GAT2(n_heads = 1) #, GraphSage(n_layers = n_layers), FCL() #
path =  './models/GATSY.pt'#'models/GAT2_nobatch.pt' #'models/three_layerSAGElstm.pt'   #Choose the path where to save the trained model
first = True

''' NeighborLoader is really important when it comes to train with mini-batches, it provides easily the splittings and the training is faster. '''

' NeighborLoader is really important when it comes to train with mini-batches, it provides easily the splittings and the training is faster. '

In [23]:
model_name = 'GAT2'

n_heads = 1

lr = 6e-5

n_layers = 2
weight_decay = 1e-2
num_epochs = 50
model = GAT2(n_heads).to('cpu') # GraphSage(), FCL()
# visualize(model(X.to('cpu'), A.to('cpu')), color = labels, dim = 3)
# visualize(model(X.to('cpu'), A.to('cpu')), color = labels, dim = 2)


# model.to('cuda')
# X.to('cuda')
# A.to('cuda')
loss_mode = 'triplet+'

if loss_mode == 'triplet' or loss_mode == 'triplet+':
    loss = torch.nn.TripletMarginLoss(margin=0.2)
elif loss_mode == 'sphere':
    loss = losses.SphereFaceLoss(num_classes, 
                    256, 
                    margin=24, 
                    scale=1)

train_loader = NeighborLoader(data, input_nodes = data.vtrain_mask, num_neighbors=[25]*n_layers, shuffle = True, batch_size = 512)
test_loader = NeighborLoader(data, input_nodes = data.val_mask, num_neighbors=[25]*n_layers, shuffle = False, batch_size = 512)

    

optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = weight_decay)
scheduler=lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min= 0, last_epoch= -1, verbose=True)

path =  './models/GATSY.pt'#'./models/best_gat_random.pt'
trainer = Trainer(model.to(device), optimizer, scheduler, loss, num_epochs, mode, path, first, loss_mode)
accuracy = trainer.train()  # This cell make the training start.




/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/torch_geometric/sampler/neighbor_sampler.py:50: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


Adjusting learning rate of group 0 to 6.0000e-05.
Processing 1-th epoch
Training step....
Testing step....
Evaluation step....
Adjusting learning rate of group 0 to 5.9941e-05.
At the 1-th epoch we have obtained: train_loss 2.483031 	 test_loss 0.022294 	 test_loss_pred 2.415892 	 test_accuracy 0.520901 	 f1_score 0.183740
Processing 2-th epoch
Training step....
Testing step....
Evaluation step....
Adjusting learning rate of group 0 to 5.9763e-05.
At the 2-th epoch we have obtained: train_loss 2.441967 	 test_loss 0.021225 	 test_loss_pred 2.617049 	 test_accuracy 0.526116 	 f1_score 0.261134
Processing 3-th epoch
Training step....
Testing step....
Evaluation step....
Adjusting learning rate of group 0 to 5.9469e-05.
At the 3-th epoch we have obtained: train_loss 2.593603 	 test_loss 0.018263 	 test_loss_pred 2.718917 	 test_accuracy 0.532141 	 f1_score 0.276617
Processing 4-th epoch
Training step....
Testing step....
Evaluation step....
Adjusting learning rate of group 0 to 5.9057e-05

KeyboardInterrupt: 

In [ ]:
model.to('cpu')
X = X.to('cpu')
A = A.to('cpu')
labels = labels.to('cpu')


In [15]:
visualize(model(X, A).detach(), color = labels)


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


(array([[-7.3011742e+00, -2.3943777e+01],
        [-1.1953631e+01, -8.8758209e+01],
        [-1.6378843e+01,  3.3418159e+01],
        ...,
        [-2.5010901e-02, -3.6453400e+01],
        [ 1.6870333e+01,  3.5130085e+01],
        [ 3.1312304e+01, -7.1890198e+01]], dtype=float32),
               x1         x2          y                  name  \
 0      -7.301174 -23.943777      dance  Tracy + the Plastics   
 1     -11.953631 -88.758209      metal             Amesoeurs   
 2     -16.378843  33.418159       soul          Eddie Holman   
 3       0.954631  22.557329       folk             Manu Chao   
 4     -30.680714 -23.779425      dance          Hot Banditoz   
 ...          ...        ...        ...                   ...   
 11256  36.584526  -1.486688       rock      The Spill Canvas   
 11257  45.465332 -42.381046  hard rock             Scorpions   
 11258  -0.025011 -36.453400      indie        Beans on Toast   
 11259  16.870333  35.130085       rock          Elkie Brooks   
 11

In [16]:
visualize(model(X, A).detach(), color = labels, dim = 3)


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



(array([[  2.2904575,   8.724335 ,   2.19167  ],
        [ 10.561589 ,  45.589165 ,  -0.4055735],
        [-17.03577  , -10.01962  ,   8.287021 ],
        ...,
        [ 11.160269 ,  17.118326 ,   2.8582115],
        [ -5.8231597,  -7.8293967,  -9.3865185],
        [ 26.744165 ,  29.580872 , -11.19025  ]], dtype=float32),
               x1         x2         x3          y                  name  \
 0       2.290457   8.724335   2.191670      dance  Tracy + the Plastics   
 1      10.561589  45.589165  -0.405573      metal             Amesoeurs   
 2     -17.035770 -10.019620   8.287021       soul          Eddie Holman   
 3       7.190673 -11.566864   5.264207       folk             Manu Chao   
 4      -9.633234  13.600788  15.779529      dance          Hot Banditoz   
 ...          ...        ...        ...        ...                   ...   
 11256  -9.001769  -0.205375 -17.018017       rock      The Spill Canvas   
 11257  -0.605174  21.173586 -14.878643  hard rock             Scorp

In [ ]:
model.to('cpu')

In [21]:
X = X.to('cpu')
A = A.to('cpu')
embedding = model(X, A)

In [ ]:
losss(embedding, labels)